In [1]:
import rasterio

import os
import numpy as np
import glob
import csv
import cv2
import random

In [2]:
cities = os.listdir('../../datasets/onera/train_labels/')
cities.sort()
input_size = 64

In [6]:
total_change = 0
total_nochange = 0
train_change = []
train_nochange = []
for city in cities[4:]:
    print (city)
    d = cv2.imread('../../datasets/onera/train_labels/' + city + '/cm/' + 'cm.png', 0) / 255
#     print (city, np.unique(d))
#   total_change += np.sum(d)
#   total_pixels += r.shape[0] * r.shape[1]
#     print (d.shape)
    for i in range(0, d.shape[0], 16):
        for j in range(0, d.shape[1], 16):
            if i + input_size < d.shape[0] and j + input_size < d.shape[1]:
                if np.sum(d[i:i+input_size, j:j+input_size]) >= 0:
                    train_change.append([city, i, j])
                    change = np.sum(d[i:i+input_size, j:j+input_size])
                    total_change += change
                    total_nochange += (input_size * input_size - change)

random.shuffle(train_nochange)                  
train = train_change + train_nochange[:len(train_change)]
print (total_change/total_nochange)

test = []
for city in cities[:4]:
    print (city)
    r = rasterio.open('../../datasets/onera/train_labels/' + city + '/cm/' + city + '-cm.tif')
    for i in range(0, r.shape[0], input_size):
        for j in range(0, r.shape[1], input_size):
            if i + input_size < r.shape[0] and j + input_size < r.shape[1] and \
            np.sum(d[i:i+input_size, j:j+input_size]) >= 1:
                test.append([city, i, j])

bercy
bordeaux
cupertino
hongkong
mumbai
nantes
paris
pisa
rennes
saclay_e
0.0207362949519
abudhabi
aguasclaras
beihai
beirut


/usr/local/lib/python3.5/dist-packages/rasterio/__init__.py:240: NotGeoreferencedWarning: Dataset has no geotransform set. Default transform will be applied (Affine.identity())
  s = DatasetReader(fp, driver=driver, **kwargs)


In [7]:
len(train), len(test)

(11878, 120)

In [8]:
fout = open('../../datasets/onera/train_64x64.csv', 'w')
w = csv.writer(fout)
for t in train:
    w.writerow(t)
fout.close()

fout = open('../../datasets/onera/test_64x64.csv', 'w')
w = csv.writer(fout)
for t in test:
    w.writerow(t)
fout.close()